In [ ]:
import tensorflow as tf

In [ ]:
# limit the GPU memory growth
gpu = tf.config.list_physical_devices('GPU')
print("Num GPUs Available: ", len(gpu))
if len(gpu) > 0:
    tf.config.experimental.set_memory_growth(gpu[0], True)

In [ ]:
from pathlib import Path
from transformers import BartConfig, BartTokenizer, TFBartForConditionalGeneration

model_path = Path('../bart_large_cnn')

def load_tokenizer():
    tokenizer = BartTokenizer.from_pretrained(str(model_path))
    config = BartConfig.from_pretrained(str(model_path))
    config.output_hidden_states = True
    return tokenizer, config

def load_model(config):
    model = TFBartForConditionalGeneration.from_pretrained(model_path, config=config)
    return model

In [ ]:
tokenizer, config = load_tokenizer()
model = load_model(config)

In [ ]:
sample = """..."""
print(len(sample.split()), ' words')

sentences = sample.split(". ")
concatenated = []

for sentence in sentences:
    words = sentence.split()
    if not concatenated:
        concatenated.append(sentence)
    elif len(concatenated[-1].split()) + len(words) < 640:
        concatenated[-1] += ". " + sentence
    else:
        concatenated.append(sentence)

for i, sentence in enumerate(concatenated):
    print(i, len(sentence.split()))

In [ ]:
from transformers import SummarizationPipeline

generator = SummarizationPipeline(model=model, tokenizer=tokenizer, framework='tf', device=0, batch_size=8)

In [ ]:
from datetime import datetime

time_start = datetime.now()
outputs = generator(concatenated, num_return_sequences=1, num_beams=1, top_k=50)
time_end = datetime.now()
print('Time cost: ', (time_end - time_start).seconds, ' seconds')

for i, output in enumerate(outputs):
    print("{}: {} \n\n".format(i+1, output['summary_text']))